Что нужно сделать далее:
1. Рисовать графики
2. Изменение параметров пласта
3. КПД
4. КВД
5. Непроницаемая граница
6. Постоянное давление на контуре
7. Диагностический график
8. Добавление классов для каждого графика свои данные!!!
9. Скин фактор
10. ВСС
11. Алгорит Стехфеста
12. Изменение давления в безразмерных координатах
13. Сравнение КВД и КПД
14. Гидропрослушивание
15. Логарифмическая аппроксимация имеет место при...
16. Эталонная кривая

.
.
.
99. Интепретатор ГДИС
.
.
.
999. CRM
1000. Термометрия

ИСП. Графики

Импортируем все необходимые модули

In [76]:

#для bokeh
from bokeh.layouts import column,row,layout
from bokeh.models import ColumnDataSource, Slider, RadioButtonGroup,TableColumn, DataTable
from bokeh.plotting import figure, output_file, show
from bokeh.themes import Theme
from bokeh.io import show, output_notebook, output_file 
from bokeh.layouts import widgetbox
from bokeh.models.tools import HoverTool, PointDrawTool
#прочие
import pandas as pd
import numpy as np
from scipy import special
import os
import scipy as sp
import matplotlib.pyplot as plt
import pylab




output_notebook() #для вывода в Jupyter Notebook

Loading BokehJS ...

Зададим исходные данные

In [54]:
k_md=10
mu_sp=1
ct_1atm=5*10**(-5)
rw_m=0.1
q_m3day=10
B_m3m3=1.2
Pi_bar=250
m=0.2
h_m=1

Создадим основные и вспомогательные функции

In [55]:

def out(string1, result,string2=''): #выпод вычислений с комментариями
    space='  '
    print(string1 + space + str(result) + space + string2 + '\n')


#Инегральная экспонента и логарифмическая аппроксимация

Ei=sp.special.expi  #f=-Ei(-x), используется в уравнении решения фильтрации
E1=sp.special.exp1 #f=E1(x);   E1(x)=-Ei(-x)=-ln(x)-gammd
ln=np.log

def LogApprox(x): #логарифмическая аппроксимация, только для числа, не списка
    gamma=0.57
    #gamma=0.809
    #result=(-1)*math.log(x)-gamma
    result=(-1)*ln(x)-gamma
    return result


def genvaluesx(k,x0=0.001): #генерация значений x c множителем к
    values=[float(x0)] #начальное значение
    for i in range(20): #число элементов 
        values.append(values[i]*k)
    result=np.asanyarray(values)
    return result

def save(name='', fmt='png'): #в текущей директории должна быть папка pictures
    pwd = os.getcwd()
    iPath = './{}'.format(fmt)
    if not os.path.exists(iPath):
        os.mkdir(iPath)
    os.chdir(iPath)
    plt.savefig('{}.{}'.format(name, fmt), fmt='png')
    os.chdir(pwd)
    

def td(t,k=k_md,m=m,mu=mu_sp,ct=ct_1atm,rw=rw_m): #безразмерное время
    result=0.00036*k*t/(m*mu*ct*rw*rw)
    return result

def pd_E1(rd,td): #безразмерное давление через интегрально экспоненциальную функцию
    result=1/2*E1(rd**2/4/td)
    return result

#??? какая из функций
def pd_ln(rd,td): #безравзерное давление через логарифм
    #result=1/2*LogApprox(rd**2/4/td)
    result=1/2*(ln(td/rd**2)+0.80907)
    return result

def rd(r,rw=rw_m): #безразмерное расстояние
    result=r/rw
    return result

def pwf(pd,k=k_md,h=h_m,q=q_m3day,b=B_m3m3,mu=mu_sp,pi=Pi_bar): #преобразование безразмерного давления в размерное
    result=pi-18.41*pd*q*b*mu/k/h
    return result

   
def p(r,t,mod=1): #решение линейного стока 
    td1=td(t)
    rd1=rd(r)
    if mod==1: #решение линейного стока с помощью интегральной экспоненты
        pd1=pd_E1(rd1,td1)
        result=pwf(pd1)
        return result
    if mod==2: #решение линейного стока с помощью логарфима
        pd1=pd_ln(rd1,td1)
        result=pwf(pd1)
        return result       

def mistake(a,b): #вычисление относительной ошибки
    result=(a-b)/a*100    
    return np.absolute(result)
    


 


Выбор типа осей для построения ВСЕХ графиков. Для активации необходимо перезапустить ячейку с графиком

In [56]:
#параметры, устанавливающие тип осей для ВСЕХ графиков, меняются через функцию(кнопки)
ytype='linear' 
xtype='linear'
def choiceaxis(): #вывод кнопок для смены типов осей
    def modify_doc(doc):
        def callback2(attr, old, new):
            global xtype
            global ytype 
            if new==0:
                ytype='linear'
                xtype='linear'
            if new==1:
                ytype='linear'
                xtype='log'
            if new==2:
                ytype='log'
                xtype='linear'       
            if new==3:
                ytype='log'
                xtype='log'

        button_group = RadioButtonGroup(labels=["y=f(x)", "y=ln(f(x))", "ln(y)=f(x)",'ln(y)=ln(f(x))'], active=0)
        button_group.on_change('active', callback2)
        doc.add_root(button_group)       
    show(modify_doc)
choiceaxis()

Интегральная экспонента  и ее логарифмическая аппроксимация

In [57]:
#исходные данные для построения графика
valuesx=genvaluesx(1.1)
data33 = {'x_values': valuesx,
        'y_values': E1(valuesx),
        'y_values2': LogApprox(valuesx),
         'mistake': mistake(E1(valuesx),LogApprox(valuesx))}

data2 = pd.DataFrame(data33)

def modify_doc(doc):
    source2 = ColumnDataSource(data=data2)    #место, где хранятся данные фигуры
    plot1 = figure(title="E и Ln", x_axis_label='x', y_axis_label='E1(x) и Ln(x)',x_axis_type=xtype, y_axis_type=ytype)
    def plotline():
        plot1.line('x_values', 'y_values', color="red", source=source2, legend='E(x)',name='E(x)')
        plot1.line('x_values', 'y_values2', color="blue", source=source2, legend='-ln(x)-0.57',name='Ln(x)')
        #plot1.line('x_values', 'mistake', color="green", source=source2, legend='Ошибка',name='Ошибка')
    plotline()
    def callback(attr, old, new): #при выполнении события для слайдера
        data=data2.copy()
        if new == 0:
            data=data2.copy()
        else:
            valuesx=genvaluesx(new)
            data['x_values']=valuesx
            data['y_values']=E1(valuesx)
            data['y_values2']=LogApprox(valuesx)       
        source2.data = ColumnDataSource(data=data).data  
    slider = Slider(start=1.1, end=2, value=1.1, step=0.1, title="Множитель к, расширяющий ось x") #отображение слайдера
    slider.on_change('value', callback) #события для слайдера, вызывающее функции
    doc.add_root(layout([[slider],[plot1]]))
    
show(modify_doc)

Решение линейного стока p(t)

In [58]:
choiceaxis()

In [59]:
t0=2*10**(-7)
valuesx=genvaluesx(1.3,t0)

data33 = {'x_values': valuesx,
        'y_values':p(0.1,valuesx),
        'y_values2': p(0.1,valuesx,2),
         'mistake': mistake(E1(valuesx),LogApprox(valuesx))}

data2 = pd.DataFrame(data33)

def modify_doc(doc):
    source2 = ColumnDataSource(data=data2)  
    plot1 = figure(title="P(t), r=0.1", x_axis_label='t', y_axis_label='P',x_axis_type=xtype, y_axis_type=ytype)
    def plotline():
        plot1.line('x_values', 'y_values', color="red", source=source2, legend='через E(x)',name='E(x)')
        plot1.line('x_values', 'y_values2', color="blue", source=source2, legend='через ln(x)',name='Ln(x)')
    plotline()
    def callback(attr, old, new):
        data=data2.copy()
        if new == 0:
            data=data2.copy()
        else:
            valuesx=genvaluesx(new,t0)
            data['x_values']=valuesx
            data['y_values']=p(0.1,valuesx)
            data['y_values2']=p(0.1,valuesx,2)
        source2.data = ColumnDataSource(data=data).data  
    slider = Slider(start=1.1, end=2, value=1.3, step=0.1, title="Множитель к, расширяющий ось x")
    slider.on_change('value', callback)
    doc.add_root(layout([[slider],[plot1]]))

    
show(modify_doc)

In [60]:
choiceaxis()

Решение линейного стока #3 p(r,t)

In [61]:
r0=0.1
t=1
valuesx=genvaluesx(1.3,r0)
#ymistake=mistake(E1(valuesx),LogApprox(valuesx))
data33 = {'x_values': valuesx,
        'y_values':p(valuesx,t),
        'y_values2': p(valuesx,t,2),
         'mistake': mistake(E1(valuesx),LogApprox(valuesx)),
         'initialp': valuesx*0+Pi_bar}

data2 = pd.DataFrame(data33)

def modify_doc(doc):
    source2 = ColumnDataSource(data=data2)  
    plot1 = figure(title="P(r,t), rw=0.1", x_axis_label='r,м', y_axis_label='P, атм',x_axis_type=xtype, y_axis_type=ytype)
    
    plot1.line('x_values', 'y_values', color="red", source=source2, legend='через E(x)',name='E(x)')
    plot1.line('x_values', 'y_values2', color="blue", source=source2, legend='через ln(x)',name='Ln(x)')
    plot1.line('x_values', 'initialp', color="green", source=source2, legend='Pi',name='Pi')
    
    def callback(attr, old, new):
        data=data2
        if new == 0:
            data=data2.copy()
        else:
            valuesx=genvaluesx(new,r0)
            data['initialp']=valuesx*0+Pi_bar
            data['x_values']=valuesx
            data['y_values']=p(valuesx,t)
            data['y_values2']=p(valuesx,t,2)
        source2.data = ColumnDataSource(data=data).data  
    
    def callback2(attr, old, new):
        data=data2
        if new == 0:
            data=data2.copy()
        else:
            data['y_values']=p(data['x_values'],new)
            data['y_values2']=p(data['x_values'],new,2)
        source2.data = ColumnDataSource(data=data).data  
    
    slider = Slider(start=1.1, end=2, value=1.3, step=0.05, title="Множитель к, расширяющий ось x")
    slider.on_change('value', callback)
    slidert = Slider(start=1/(10**3), end=100, value=0.01, step=0.01, title="Время")
    slidert.on_change('value', callback2)
    doc.add_root(layout([[slider,slidert],[plot1]]))

    
show(modify_doc)

In [79]:
r0=0.1
t=1
valuesx=genvaluesx(1.3,r0)
#ymistake=mistake(E1(valuesx),LogApprox(valuesx))
data33 = {'x_values': valuesx,
        'y_values':p(valuesx,t),
        'y_values2': p(valuesx,t,2),
         'mistake': mistake(E1(valuesx),LogApprox(valuesx)),
         'initialp': valuesx*0+Pi_bar,
         'x':valuesx*0,
         'y':valuesx*0}

data2 = pd.DataFrame(data33)

def modify_doc(doc):
    source2 = ColumnDataSource(data=data2)  
    plot1 = figure(title="P(r,t), rw=0.1", x_axis_label='r,м', y_axis_label='P, атм',x_axis_type=xtype, y_axis_type=ytype)
    
    plot1.line('x_values', 'y_values', color="red", source=source2, legend='через E(x)',name='E(x)')
    plot1.line('x_values', 'y_values2', color="blue", source=source2, legend='через ln(x)',name='Ln(x)')
    plot1.line('x_values', 'initialp', color="green", source=source2, legend='Pi',name='Pi')
    r1 = plot1.rect('x', 'y', 0.1, 0.1, source=source2)

    plot1.add_tools(PointDrawTool(renderers=[r1]))
    columns=[TableColumn(field='x',title='x'),
            TableColumn(field='y',title='y')]
    table=DataTable(source=source2,columns=columns, editable=True)
    def callback(attr, old, new):
        data=data2
        if new == 0:
            data=data2.copy()
        else:
            valuesx=genvaluesx(new,r0)
            data['initialp']=valuesx*0+Pi_bar
            data['x_values']=valuesx
            data['y_values']=p(valuesx,t)
            data['y_values2']=p(valuesx,t,2)
        source2.data = ColumnDataSource(data=data).data  
    
    def callback2(attr, old, new):
        data=data2
        if new == 0:
            data=data2.copy()
        else:
            data['y_values']=p(data['x_values'],new)
            data['y_values2']=p(data['x_values'],new,2)
        source2.data = ColumnDataSource(data=data).data  
    
    slider = Slider(start=1.1, end=2, value=1.3, step=0.05, title="Множитель к, расширяющий ось x")
    slider.on_change('value', callback)
    slidert = Slider(start=1/(10**3), end=100, value=0.01, step=0.01, title="Время")
    slidert.on_change('value', callback2)
    doc.add_root(layout([[slider,slidert],[plot1],[table]]))
    
    
show(modify_doc)

In [47]:
a=[0,0,0]
b=[0,0,0]
c=[0,1,2]
d=[0,1,2]
r0=0.1
t=1
valuesx=genvaluesx(1.3,r0)
data33 = {'x':a,
         'y':b,
         'y2':c,
         'x2':d}

data2 = pd.DataFrame(data33)

source = ColumnDataSource(data2) 
p=figure()
p.rect('x2','y2',0.1, 0.1,source=source)
#c1 = p.circle('x', 'y', source=source)
r1 = p.rect('x', 'y', 0.1, 0.1, source=source)
#tool = PointDrawTool(renderers=[c1, r1])
#plot = figure(tools="pan,wheel_zoom,box_zoom,reset")
p.add_tools(PointDrawTool(renderers=[r1]))
show(p)

def modify_doc2(doc22):
    source2 = ColumnDataSource(data=data2)  
    plot1 = figure(title="P(r,t), rw=0.1", x_axis_label='r,м', y_axis_label='P, атм',x_axis_type=xtype, y_axis_type=ytype)
    
    plot1.line('x', 'y', color="red", source=source2, legend='через E(x)',name='E(x)')
    r1 = plot1.rect('x', 'y', 0.1, 0.1, source=source)

    p.add_tools(PointDrawTool(renderers=[r1]))
   
    '''def callback(attr, old, new):
        data=data2
        if new == 0:
            data=data2.copy()
        else:
            valuesx=genvaluesx(new,r0)
            data['initialp']=valuesx*0+Pi_bar
            data['x_values']=valuesx
            data['y_values']=p(valuesx,t)
            data['y_values2']=p(valuesx,t,2)
        source2.data = ColumnDataSource(data=data).data  
    
    def callback2(attr, old, new):
        data=data2
        if new == 0:
            data=data2.copy()
        else:
            data['y_values']=p(data['x_values'],new)
            data['y_values2']=p(data['x_values'],new,2)
        source2.data = ColumnDataSource(data=data).data  '''
    
    doc22.add_root(layout([[plot1]]))
    

    
show(modify_doc2)

ERROR:tornado.application:Uncaught exception GET /autoload.js?bokeh-autoload-element=6031&bokeh-absolute-url=http://localhost:58530&resources=none (::1)
HTTPServerRequest(protocol='http', host='localhost:58530', method='GET', uri='/autoload.js?bokeh-autoload-element=6031&bokeh-absolute-url=http://localhost:58530&resources=none', version='HTTP/1.1', remote_ip='::1')
Traceback (most recent call last):
  File "C:\ProgramData\PPythonAnacondaOneLove\lib\site-packages\tornado\web.py", line 1592, in _execute
    result = yield result
  File "C:\ProgramData\PPythonAnacondaOneLove\lib\site-packages\tornado\gen.py", line 1133, in run
    value = future.result()
  File "C:\ProgramData\PPythonAnacondaOneLove\lib\site-packages\tornado\gen.py", line 1141, in run
    yielded = self.gen.throw(*exc_info)
  File "C:\ProgramData\PPythonAnacondaOneLove\lib\site-packages\bokeh\server\views\autoload_js_handler.py", line 60, in get
    session = yield self.get_session()
  File "C:\ProgramData\PPythonAnaconda

In [40]:
print(source.data)

{'index': array([0, 1, 2], dtype=int64), 'x': array([0, 0, 0], dtype=int64), 'y': array([0, 0, 0], dtype=int64), 'y2': array([0, 1, 2], dtype=int64), 'x2': array([0, 1, 2], dtype=int64)}


Полигон для проверки функций #3

In [ ]:
data33 = {'x_values': valuesx,
        'y_values': solvE1}
data2 = pd.DataFrame(data33)

def modify_doc(doc):
   
    source2 = ColumnDataSource(data=data2)
    output_notebook()
    plot1 = figure(title="P(t), r=0.1 м", x_axis_label='t, часы', y_axis_label='P по Е1')
    plot1.line('x_values', 'y_values', source=source2)

    def callback(attr, old, new):
        data=data2.copy()
        if new == 0:
            data=data
        else:
            valuesx=genvaluesx(new)
            valuesx=np.asanyarray(valuesx)
            solvE1=p(0.1,valuesx)
            data333 = {'x_values': valuesx,'y_values': solvE1}
            data22 = pd.DataFrame(data333)
            
            data['x_values']=data22['x_values']
            data['y_values']=data22['y_values']
        source2.data = ColumnDataSource(data=data).data
    
    slider = Slider(start=1, end=2, value=1.1, step=0.1, title="Множитель к, расширяющий ось x")
    slider.on_change('value', callback)
    
    doc.add_root(column(slider, plot1))
    


    
show(modify_doc)

In [ ]:

x=0.001
k=1.4 #множитель по оси x


td1=td(83.336111111)
rd1=rd(0.1)

valuesx=genvaluesx(k)
valuesx=np.asanyarray(valuesx)
print(p(0.1,0.00001))
print(p(0.1,0.00001,2))
solvE1=p(0.1,valuesx)
solvLog=p(0.1,valuesx,2)

out('-Ei(-x)=',-Ei(-x), 'Пример значения интегральной экспоненты при x={}'.format(str(x)))
out('E1(x)=',E1(x), 'E1(x)=-Ei(-x)=-ln(x)-gamma при x={}'.format(str(x)))
out('-ln(x)-gamma=',LogApprox(x),'логарифмическая аппроксимация интегральной экспоненты при x={}'.format(str(x)))


#figsize = (4,3)
#fig=plt.figure(figsize=figsize, dpi=150, facecolor='green', frameon=True)
fig=plt.figure()
#plt.xscale('log') 
plt.plot(valuesx,solvE1, label='E1')
plt.plot(valuesx,solvLog, label='Ln')

plt.title('Интегральная экспонента и ее аппроксимация')   # заголовок
plt.xlabel('x')   # подпись оси OX
plt.ylabel('E1(x); -ln(x)-0.57')   # подпись оси OY
plt.legend()
plt.grid(True)

save('pic_1')
plt.show()